In [1]:
### Load CSV ###

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

df = pd.read_csv('csgo_round_snapshots.csv')

In [2]:
### Prepare Data ###

dfCategorical = pd.get_dummies(df)

dfCategorical['bomb_planted'] = dfCategorical['bomb_planted'].astype(int) # Replace boolean column by numbers

# Drop useless columns (avoid data redundancy or column with unique value)
dfCategorical = dfCategorical.drop(['map_de_vertigo', 'round_winner_T', 'ct_weapon_bizon', 'ct_weapon_g3sg1', 't_weapon_m249', 'ct_weapon_negev', 'ct_weapon_r8revolver', 'ct_weapon_sawedoff'], axis=1)

y = dfCategorical['round_winner_CT']
X = dfCategorical.drop('round_winner_CT', axis=1)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [3]:
### Model ###

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

#params = {'max_depth': np.arange(1, 10)}
params = {'n_neighbors': np.arange(2, 9, 2)}

model = KNeighborsClassifier()

gscv = GridSearchCV(model, params)

%time gscv.fit(X_train, y_train)

print(gscv.best_params_, " ", gscv.best_score_)

y_pred = gscv.predict(X_test)

CPU times: user 1h 2min 23s, sys: 13 s, total: 1h 2min 37s
Wall time: 1h 2min 53s
{'n_neighbors': 2}   0.8263564059105809


In [4]:
### Metrics ###

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(cross_val_score(model, X_train, y_train, cv=5))

[[17654  1015]
 [ 4679 13375]]
              precision    recall  f1-score   support

           0       0.79      0.95      0.86     18669
           1       0.93      0.74      0.82     18054

    accuracy                           0.84     36723
   macro avg       0.86      0.84      0.84     36723
weighted avg       0.86      0.84      0.84     36723

[0.81129653 0.81374723 0.81612884 0.81000175 0.81402813]
